# Nitsche's Method for boundary and interface conditions

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw

mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

In [ ]:
fes = H1(mesh, order=4)
u,v = fes.TnT()

h = specialcf.mesh_size
a = BilinearForm(grad(u)*grad(v)*dx + 1/h*u*v*ds).Assemble()
f = LinearForm(10*v*dx).Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse() * f.vec

print ("error bc:", Integrate((gfu-0)**2, mesh.Boundaries(".*")))
Draw (gfu);

In [ ]:
n = specialcf.normal(mesh.dim)

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx + 1/h*u*v*ds
a += (-n*grad(u)*v-n*grad(v)*u) * ds(skeleton=True)
a.Assemble()

gfu.vec.data = a.mat.Inverse() * f.vec

print ("error bc:", Integrate((gfu-0)**2, mesh.Boundaries(".*")))
Draw (gfu);

## Interfaces

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw

In [ ]:
square = MoveTo(0,0).Rectangle(1,1).Face()
circo = Circle((0.5,0.5), 0.3).Face()
circ = Circle((0.5,0.5), 0.3).Face()
bar = MoveTo(0.3,0.45).Rectangle(0.4,0.1).Face()

square.edges.name="outer"
circ.edges.name="gammai"
circo.edges.name="gammao"
outer = square-circo
outer.faces.name = "outer"

circ.faces.name = "inner"
bar.faces.name = "bar"
inner = circ-bar

both = Compound([outer, inner, bar])
mesh = Mesh(OCCGeometry(both, dim=2).GenerateMesh(maxh=0.05)).Curve(3)
print (mesh.GetBoundaries())
Draw (mesh);

In [ ]:
def SetMeshRotation(angle):
    mesh.UnsetDeformation()
    deform = GridFunction(VectorH1(mesh, order=3))

    rotmat = CF( (cos(angle), -sin(angle), sin(angle), cos(angle))).Reshape( (2,2))
    center = CF( (0.5, 0.5) )
    pos = CF( (x,y) )

    deform.Set( (rotmat-Id(2))*(pos-center), definedon=mesh.Materials("inner|bar"))

    mesh.SetDeformation(deform) 
    return deform

In [ ]:
from time import sleep
scene = Draw (mesh)

for i in range(10):
    SetMeshRotation(i/10)
    scene.Redraw()
    sleep(0.1)

In [ ]:
fes = H1(mesh, order=3, dirichlet="outer")
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx + u*v*dx).Assemble()
f = LinearForm((x-0.5)*v*dx("bar")).Assemble()
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs())*f.vec
Draw (gfu);

In [ ]:
deform = SetMeshRotation(0.8)
mesh.UnsetDeformation()

a = BilinearForm(grad(u)*grad(v)*dx + u*v*dx)

contact = ContactBoundary(mesh.Boundaries("gammai"), mesh.Boundaries("gammao"))
h = specialcf.mesh_size
contact.AddIntegrator (100/h*(u-u.Other()) * (v-v.Other()))
# consisteny term not yet implemented for contact boundary
# contact.AddIntegrator (n*grad(u)*(v.Other()-v)+n*grad(v)*(u.Other()-u))
contact.Update (deform, bf=a)

gfu.vec[:] = 0
a.AssembleLinearization(gfu.vec)
f = LinearForm(1e3*(x-0.5)*v*dx("bar")).Assemble()

gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs())*f.vec
mesh.SetDeformation(deform)

Draw (gfu)
Draw (grad(gfu), mesh);

# Hybrid Interfaces